Author:
        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

# Airline Passenger Satisfaction Classification

https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction/data


<p>
    <img src="https://www.travelandleisure.com/thmb/h97kSvljd2QYH2nUy3Y9ZNgO_pw=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/plane-data-BUSYROUTES1217-f4f84b08d47f4951b11c148cee2c3dea.jpg" width=600>
</p>

## 1. Load Libraries

In [1]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

path_append = "../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [2]:
import torch
import pandas as pd

## 2. Load  Dataset

In [3]:
df_train = pd.read_csv(path_append + '../data/Airline Customer Satisfaction/train.csv')
df_test = pd.read_csv(path_append + '../data/Airline Customer Satisfaction/test.csv')
df_train.head()
df_test.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,...,5,5,5,5,2,5,5,50,44.0,satisfied
1,1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,...,4,4,4,4,3,4,5,0,0.0,satisfied
2,2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,...,2,4,1,3,2,2,2,0,0.0,neutral or dissatisfied
3,3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,...,1,1,1,1,3,1,4,0,6.0,satisfied
4,4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,...,2,2,2,2,4,2,4,0,20.0,satisfied


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  object 
 3   Customer Type                      103904 non-null  object 
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  object 
 6   Class                              103904 non-null  object 
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103904 non-null  int64  
 11  Gate location                      1039

In [5]:
# Check labels count
df_train[['satisfaction']].value_counts()

satisfaction           
neutral or dissatisfied    58879
satisfied                  45025
Name: count, dtype: int64

## 3. Preprocessing

In [6]:
from tools.preprocessing.data_frame import process_dataframe

target_columns = ['satisfaction']
drop_columns = ['Unnamed: 0','id']

# Assuming df_train and df_test are your initial dataframes
df_train_length = len(df_train)
df_test_length = len(df_test)

# Concatenate the dataframes
df = pd.concat([df_train, df_test], axis=0)

# Process the combined dataframe
df, description_train = process_dataframe(df, target_columns, drop_columns=drop_columns)

# Split the dataframe back into training and test sets
df_train = df.iloc[:df_train_length]
df_test = df.iloc[df_train_length:]

num_features = description_train['num_features']
num_classes = description_train['num_classes']

print(f"Number of features after scaling: {num_features}")
print(f"Number of classes after scaling: {num_classes}")
description_train

Column 'Gender' has 2 unique values.
Column 'Customer Type' has 2 unique values.
Column 'Type of Travel' has 2 unique values.
Column 'Class' has 3 unique values.
Column 'satisfaction' has 2 unique values.
Number of features after scaling: 24
Number of classes after scaling: 1


{'num_features': 24,
 'num_classes': 1,
 'encoded_columns': Index(['Gender', 'Customer Type', 'Type of Travel', 'Class'], dtype='object'),
 'target_encoded_columns': Index(['satisfaction'], dtype='object'),
 'scalers': {'Age': 'minmax',
  'Arrival Delay in Minutes': 'robust',
  'Baggage handling': 'standard',
  'Checkin service': 'robust',
  'Cleanliness': 'minmax',
  'Departure Delay in Minutes': 'robust',
  'Departure/Arrival time convenient': 'minmax',
  'Ease of Online booking': 'minmax',
  'Flight Distance': 'robust',
  'Food and drink': 'minmax',
  'Gate location': 'minmax',
  'Inflight entertainment': 'minmax',
  'Inflight service': 'standard',
  'Inflight wifi service': 'minmax',
  'Leg room service': 'minmax',
  'On-board service': 'minmax',
  'Online boarding': 'minmax',
  'Seat comfort': 'minmax'},
 'target_scalers': {}}

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Gender                             103904 non-null  float64
 1   Customer Type                      103904 non-null  float64
 2   Age                                103904 non-null  float64
 3   Type of Travel                     103904 non-null  float64
 4   Flight Distance                    103904 non-null  float64
 5   Inflight wifi service              103904 non-null  float64
 6   Departure/Arrival time convenient  103904 non-null  float64
 7   Ease of Online booking             103904 non-null  float64
 8   Gate location                      103904 non-null  float64
 9   Food and drink                     103904 non-null  float64
 10  Online boarding                    103904 non-null  float64
 11  Seat comfort                       1039

In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25583 entries, 103904 to 129486
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             25583 non-null  float64
 1   Customer Type                      25583 non-null  float64
 2   Age                                25583 non-null  float64
 3   Type of Travel                     25583 non-null  float64
 4   Flight Distance                    25583 non-null  float64
 5   Inflight wifi service              25583 non-null  float64
 6   Departure/Arrival time convenient  25583 non-null  float64
 7   Ease of Online booking             25583 non-null  float64
 8   Gate location                      25583 non-null  float64
 9   Food and drink                     25583 non-null  float64
 10  Online boarding                    25583 non-null  float64
 11  Seat comfort                       25583 non-nul

In [9]:
# Custom Dataset Class
class AirlineDataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        vals = torch.tensor(self.x[index], dtype = torch.float32)
        label = torch.tensor(self.y[index], dtype= torch.long)
        return vals, label

In [10]:

# Split the data into train, validation and test sets
X_train, y_train = df_train.iloc[:, :-1].values, df_train.iloc[:, -1:].values
X_test, y_test = df_test.iloc[:, :-1].values, df_test.iloc[:,-1:].values

trainset = AirlineDataset(X_train, y_train)
testset = AirlineDataset(X_test, y_test)

print(f"Labeled Trainset Shape: {len(trainset)}, {trainset.x.shape[1]}")
print(f"Labeled Testset Shape: {len(testset)}, {testset.x.shape[1]}")

Labeled Trainset Shape: 103904, 24
Labeled Testset Shape: 25583, 24


In [11]:
num_features =  trainset.x.shape[1]
num_classes =  trainset.y.shape[1]

num_features, num_classes 

(24, 1)

# Training

In [12]:
from tools.setting.data_config import DataConfig
from tools.setting.ml_params import MLParameters
from trainer_hub import TrainerHub

# Set the data configuration
data_config = DataConfig(dataset_name = 'airline_satisfaction', task_type='binary_classification', obs_shape=[num_features], label_size=num_classes)

#  Set the ML parameters
ml_params = MLParameters(ccnet_network = 'tabnet', encoder_network = 'none')
ml_params.algorithm.error_function = 'mae'

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=True, use_wandb=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: causal-rl. Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (.\..\saved\airline_satisfaction\causal_learning)... Done. 0.0s


In [13]:
trainer_hub.train(trainset, testset)

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Iterations:   0%|          | 0/1623 [00:00<?, ?it/s]

[0/100][50/1623][Time 9.06]
Unified LR across all optimizers: 0.0001995308238189185
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.3626	Gen: 0.6848	Rec: 0.6879	E: 0.3595	R: 0.3657	P: 1.0101
--------------------Test Metrics------------------------
accuracy: 0.7891
precision: 0.8158
recall: 0.8267
f1_score: 0.8212

[0/100][100/1623][Time 7.86]
Unified LR across all optimizers: 0.00019907191565870155
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.1801	Gen: 0.3743	Rec: 0.3705	E: 0.1840	R: 0.1763	P: 0.5647
--------------------Test Metrics------------------------
accuracy: 0.8398
precision: 0.8333
recall: 0.9060
f1_score: 0.8682

[0/100][150/1623][Time 8.09]
Unified LR across all optimizers: 0.00019861406295796434
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.1399	Gen: 0.3029	Rec: 0.3021	E: 0.1406	R: 0.1391	P: 0.4652
--------------------Test Metrics------------------------
accur

Iterations:   0%|          | 0/1623 [00:00<?, ?it/s]

[1/100][27/1623][Time 7.95]
Unified LR across all optimizers: 0.00018535742845428288
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0378	Gen: 0.1233	Rec: 0.1225	E: 0.0386	R: 0.0369	P: 0.2080
--------------------Test Metrics------------------------
accuracy: 0.8750
precision: 0.8732
recall: 0.8986
f1_score: 0.8857

[1/100][77/1623][Time 7.89]
Unified LR across all optimizers: 0.00018493111819882223
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0423	Gen: 0.1253	Rec: 0.1247	E: 0.0429	R: 0.0417	P: 0.2077
--------------------Test Metrics------------------------
accuracy: 0.9141
precision: 0.9412
recall: 0.9014
f1_score: 0.9209

[1/100][127/1623][Time 7.87]
Unified LR across all optimizers: 0.00018450578842974107
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0399	Gen: 0.1209	Rec: 0.1193	E: 0.0415	R: 0.0383	P: 0.2003
--------------------Test Metrics------------------------
accur

...

[19/100][13/1623][Time 7.86]
Unified LR across all optimizers: 4.830699201847056e-05
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0107	Gen: 0.0521	Rec: 0.0512	E: 0.0117	R: 0.0098	P: 0.0925
--------------------Test Metrics------------------------
accuracy: 0.9375
precision: 0.9247
recall: 0.9643
f1_score: 0.9441

[19/100][63/1623][Time 8.04]
Unified LR across all optimizers: 4.819588901990364e-05
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0107	Gen: 0.0507	Rec: 0.0501	E: 0.0114	R: 0.0101	P: 0.0900
--------------------Test Metrics------------------------
accuracy: 0.9023
precision: 0.8987
recall: 0.9404
f1_score: 0.9191

[19/100][113/1623][Time 8.04]
Unified LR across all optimizers: 4.8085041551142536e-05
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0110	Gen: 0.0508	Rec: 0.0501	E: 0.0116	R: 0.0103	P: 0.0900
--------------------Test Metrics------------------------
accu

Iterations:   0%|          | 0/1623 [00:00<?, ?it/s]

[20/100][40/1623][Time 7.67]
Unified LR across all optimizers: 4.4772360880466255e-05
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0115	Gen: 0.0525	Rec: 0.0519	E: 0.0121	R: 0.0110	P: 0.0928
--------------------Test Metrics------------------------
accuracy: 0.9219
precision: 0.8993
recall: 0.9640
f1_score: 0.9306

[20/100][90/1623][Time 7.66]
Unified LR across all optimizers: 4.466938730792753e-05
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0121	Gen: 0.0513	Rec: 0.0507	E: 0.0128	R: 0.0115	P: 0.0898
--------------------Test Metrics------------------------
accuracy: 0.9258
precision: 0.9096
recall: 0.9742
f1_score: 0.9408

[20/100][140/1623][Time 7.59]
Unified LR across all optimizers: 4.456665056803365e-05
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0108	Gen: 0.0488	Rec: 0.0483	E: 0.0114	R: 0.0102	P: 0.0863
--------------------Test Metrics------------------------
accu

Iterations:   0%|          | 0/1623 [00:00<?, ?it/s]

[21/100][17/1623][Time 7.80]
Unified LR across all optimizers: 4.159201831473373e-05
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0106	Gen: 0.0494	Rec: 0.0485	E: 0.0115	R: 0.0097	P: 0.0873
--------------------Test Metrics------------------------
accuracy: 0.9141
precision: 0.8973
recall: 0.9493
f1_score: 0.9225

[21/100][67/1623][Time 7.83]
Unified LR across all optimizers: 4.149635932711865e-05
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0117	Gen: 0.0489	Rec: 0.0480	E: 0.0126	R: 0.0108	P: 0.0852
--------------------Test Metrics------------------------
accuracy: 0.9062
precision: 0.8940
recall: 0.9441
f1_score: 0.9184

[21/100][117/1623][Time 7.75]
Unified LR across all optimizers: 4.1400920349069853e-05
--------------------Training Metrics--------------------
CCNet:  Three Tabnet
Inf: 0.0103	Gen: 0.0498	Rec: 0.0492	E: 0.0109	R: 0.0097	P: 0.0886
--------------------Test Metrics------------------------
accu

In [ ]:
trainer_hub.test(testset)